# [Inference] YandexCup 2022 - ML Audio Content 4th place solution
- **Training code** - https://github.com/traptrip/yandex_cup_2022_audio_4th_place
- **Offline Diffusion** - https://github.com/fyang93/diffusion

## Предсказание исполнителя трека по набору акустических признаков
### Описание задачи

На первый взгляд задача предсказания исполнителя трека выглядит странной, так как кажется, что эта информация изначально нам известна. Но при ближайшем рассмотрении, оказывается что не все так просто. Во-первых, есть задача разделения одноименных исполнителей. Когда к нам в каталог поступает новый релиз, то нам нужно как-то сопоставить исполнителя этого релиза с теми что уже есть в нашей базе и для одноименных исполнителей возникает неоднозначность. Во-вторых, и это менее очевидная часть, предсказывая исполнителей по аудио, мы неявным образом получаем модель которая выучивает похожесть исполнителей по звучанию и это также может быть полезным.

### Формат входных данных

По лицензионным соглашениям мы не можем выкладывать исходные аудио треки, поэтому в рамках данной задачи мы решили подготовить для каждого трека признаковое описание на основе аудио сигнала. Изначально выбирается случайный фрагмент трека из центральной его части (70 процентов трека) длительностью 60 секунд, если трек короче 60 секунд, то берется трек целиком. Далее, этот фрагмент разбивается на чанки размером около 1.5 секунд и шагом порядка 740 миллисекунд и затем для каждого такого чанка аудио сигнала вычисляется вектор чисел, описывающий этот чанк, размером 512, это своего рода эмбединг этого чанка. Таким образом для каждого трека мы получаем последовательность векторов или другими словами матрицу размером 512xT сохраненную в файл в виде numpy array. Во входных данных задачи есть следующие файлы:
- train_features.tar.gz
- train_features_sample.tar.gz
- test_features.tar.gz
- train_meta.tsv
- train_sample_meta.tsv
- test_meta.tsv
- compute_score.py
- naive_baseline.py
- nn_baseline.py

Первые два файла train_features.tar.gz и test_features.tar.gz это архивы с файлами, в которых хранятся признаковые описания треков обучающего и тестового подмножества соответственно.
Файл train_meta.tsv содержит отображение id треков в id исполнителей и дополнительно ссылку на относительный путь к файлу с признаковым описанием трека в архиве. Файл test_meta.tsv имеет аналогичный формат, за той лишь разницей что в нем нет id исполнителей треков. Мы отбирали треки таким образом, чтобы множества исполнителей в обучающем и тестовом подмножествах не пересекались.

Файл compute_score.py поможет вам посчитать метрику для вашего решения. Для простоты выделения валидационного подмножества треков, на котором можно оценивать метрику локально, мы разбили обучающее множество треков на 10 поддиректорий, внутри каждой из них исполнители треков также не пересекаются.

Файл naive_baseline.py содержит пример наивного решения, показывает как загружать из файла признаковые описания треков и как формировать файл с решением.

Файл nn_baseline.py содержит пример простейшего решения на основе нейросетей с использованием фреймворка pytorch

Кроме того мы добавили файлы train_features_sample.tar.gz и train_sample_meta.tsv которые содержат сэмпл данных для обучения

Все файлы можно скачать по ссылке: https://disk.yandex.ru/d/xKv1B88WtLZnPw

Альтернативно можно скачать данные (без скриптов) по ссылке https://storage.yandexcloud.net/audioml-contest22/dataset.tar.gz Сэмпл данных для обучения доступен по ссылке https://storage.yandexcloud.net/audioml-contest22/train_features_sample.tar.gz

### Формат решения

Так как исполнители в обучающем и тестовом подмножествах разные, мы не можем подходить к решению этой задачи в лоб как к задаче классификации. Поэтому, задача заключается в том чтобы для каждого трека из тестового множества, по аудио признакам трека, построить отранжированный список остальных треков из тестового множества (исключая исходных трек-запрос для которого мы строим текущий список), такой, что чем выше трек в этом списке тем более вероятно что он принадлежит тому же исполнителю что и исходный трек-запрос. Для каждого трека из тестового множества нужно вывести одну строку в итоговый файл с решением. Формат строки query_trackid <tab> trackid1 <space> trackid2 … trackid100

Качество решения мы будем мерить с помощью метрики nDCG@100 (Normalized Discounted Cumulative Gain at K, K=100)

Ссылка на википедию: https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG

Во время соревнования в лидерборде будет отображаться максимальный public score из всех ваших валидных посылок. После завершения соревнования, лидерборд будет переранжирован согласно private score вашей последней валидной посылки. Кроме этого в лидерборде будет отображаться public score посчитанный также по вашему последнему валидному решению. Это означает что он может не совпадать (в частности быть ниже) с вашим максимальным public score

In [1]:
import os
import copy
import random
from pathlib import Path
from typing import Optional

import joblib
import torch
import faiss
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
from joblib import Parallel, delayed
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [9]:
class cfg:
    SEED = 42
    CROP_SIZE = 81
    N_FOLDS = 10
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    TEST_DATA_DIR = "./data/audio/test_features"
    TEST_CSV_PATH = "./data/audio/test_meta.tsv"
    
    TRAIN_DATA_DIR = "./data/audio/train_features"
    TRAIN_CSV_PATH = "./data/audio/train_meta_with_stages.tsv"

    SUBMISSION_PATH = "submission.txt"
    CHECKPOINT_PATH = "./experiments/audio/folds_weights_f10"
    BATCH_SIZE = 512

# Seed
seed = cfg.SEED
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Utils

In [3]:
def get_ranked_list_diffusion(embeds, top_size=100):
    k = top_size + 1
    targets, embeddings = embeds
    ranks = offline_diffusion_search(embeddings, embeddings, None, truncation_size=k, kd=k)
    ranks = targets[ranks]

    ranked_list = dict()
    cnt = []
    for i, track_id in enumerate(targets):
        candidates = list(filter(lambda x: x != track_id, ranks[i, 1:]))
        ranked_list[track_id] = candidates[:100]
        cnt.append(len(ranked_list[track_id]))
    print(min(cnt), max(cnt), np.mean(cnt))

    return ranked_list


def inference(models, loader):
    all_embeddings = []
    all_track_ids = []

    for data in tqdm(loader):
        features = data["features"].to(cfg.DEVICE)
        track_ids = data["track_id"].tolist()

        embeddings = []
        for model in models:
            with torch.no_grad():
                embeddings.append(model(features))
        embeddings = torch.concat(embeddings, dim=-1)

        all_embeddings.append(embeddings.cpu())
        all_track_ids.extend(track_ids)

    all_embeddings = torch.concat(all_embeddings, dim=0)
    all_track_ids = np.array(all_track_ids)

    return all_track_ids, all_embeddings


def position_discounter(position):
    return 1.0 / np.log2(position + 1)


def get_ideal_dcg(relevant_items_count, top_size):
    dcg = 0.0
    for result_indx in range(min(top_size, relevant_items_count)):
        position = result_indx + 1
        dcg += position_discounter(position)
    return dcg


def compute_dcg(query_trackid, ranked_list, track2artist_map, top_size):
    query_artistid = track2artist_map[query_trackid]
    dcg = 0.0
    for result_indx, result_trackid in enumerate(ranked_list[:top_size]):
        assert result_trackid != query_trackid
        position = result_indx + 1
        discounted_position = position_discounter(position)
        result_artistid = track2artist_map[result_trackid]
        if result_artistid == query_artistid:
            dcg += discounted_position
    return dcg


def eval_submission(submission, gt_meta_info, top_size=100):
    track2artist_map = gt_meta_info.set_index("trackid")["artistid"].to_dict()
    artist2tracks_map = gt_meta_info.groupby("artistid").agg(list)["trackid"].to_dict()
    ndcg_list = []
    for query_trackid in tqdm(submission.keys()):
        ranked_list = submission[query_trackid]
        query_artistid = track2artist_map[query_trackid]
        query_artist_tracks_count = len(artist2tracks_map[query_artistid])
        ideal_dcg = get_ideal_dcg(query_artist_tracks_count - 1, top_size=top_size)
        dcg = compute_dcg(
            query_trackid, ranked_list, track2artist_map, top_size=top_size
        )
        try:
            ndcg_list.append(dcg / ideal_dcg)
        except ZeroDivisionError:
            continue
    return np.mean(ndcg_list)


def save_submission(submission, submission_path):
    with open(submission_path, "w") as f:
        for query_trackid, result in submission.items():
            f.write("{}\t{}\n".format(query_trackid, " ".join(map(str, result))))

# Offline Diffusion Utils

In [4]:
class BaseKNN(object):
    """KNN base class"""
    def __init__(self, embeddings: np.ndarray, ids: Optional[np.ndarray], method):
        if embeddings.dtype != np.float32:
            embeddings = embeddings.astype(np.float32)
        self.N = len(embeddings)
        self.D = embeddings[0].shape[-1]
        self.embeddings = embeddings if embeddings.flags['C_CONTIGUOUS'] \
                               else np.ascontiguousarray(embeddings)
        self.labels = ids

    def add(self, batch_size=10000):
        """Add data into index"""
        if self.N <= batch_size:
            self.index.add(self.embeddings)
        else:
            [self.index.add(self.embeddings[i:i+batch_size])
                    for i in range(0, len(self.embeddings), batch_size)]

    def search(self, queries, k=5):
        """Search
        Args:
            queries: query vectors
            k: get top-k results
        Returns:
            sims: similarities of k-NN
            ids: indexes of k-NN
        """
        if not queries.flags['C_CONTIGUOUS']:
            queries = np.ascontiguousarray(queries)
        if queries.dtype != np.float32:
            queries = queries.astype(np.float32)
        sims, ids = self.index.search(queries, k)
        ids = self.labels[ids] if self.labels is not None else ids
        return sims, ids


class KNN(BaseKNN):
    """KNN class
    Args:
        embeddings: feature vectors in database
        ids: labels of feature vectors
        method: distance metric
    """
    def __init__(self, embeddings: np.ndarray, ids: np.ndarray, method):
        super().__init__(embeddings, ids, method)
        self.index = {
            'cosine': faiss.IndexFlatIP,
            'euclidean': faiss.IndexFlatL2,
        }[method](self.D)
        if os.environ.get('CUDA_VISIBLE_DEVICES'):
            self.index = faiss.index_cpu_to_all_gpus(self.index)
        self.labels = ids
        self.add()


class ANN(BaseKNN):
    """Approximate nearest neighbor search class
    Args:
        embeddings: feature vectors in database
        ids: labels of feature vectors
        method: distance metric
    """
    def __init__(
        self, embeddings: np.ndarray, ids: Optional[np.ndarray], method="cosine", M=128, nbits=8, nlist=316, nprobe=64
    ):
        super().__init__(embeddings, ids, method)
        self.labels = ids
        self.quantizer = {
            'cosine': faiss.IndexFlatIP,
            'euclidean': faiss.IndexFlatL2
        }[method](self.D)
        self.index = faiss.IndexIVFPQ(self.quantizer, self.D, nlist, M, nbits)
        samples = embeddings[np.random.permutation(np.arange(self.N))[:self.N // 5]]
        self.index.train(samples)
        self.add()
        self.index.nprobe = nprobe


trunc_ids = None
trunc_init = None
lap_alpha = None


def get_offline_result(i):
    ids = trunc_ids[i]
    trunc_lap = lap_alpha[ids][:, ids]
    scores, _ = linalg.cg(trunc_lap, trunc_init, tol=1e-6, maxiter=20)
    return scores


def cache(filename):
    """Decorator to cache results"""

    def decorator(func):
        def wrapper(*args, **kw):
            self = args[0]
            path = os.path.join(self.cache_dir, filename)
            Path(self.cache_dir).mkdir(parents=True, exist_ok=True)
            time0 = time.time()
            if os.path.exists(path):
                result = joblib.load(path)
                cost = time.time() - time0
                # print("[cache] loading {} costs {:.2f}s".format(path, cost))
                return result
            result = func(*args, **kw)
            cost = time.time() - time0
            print("[cache] obtaining {} costs {:.2f}s".format(path, cost))
            joblib.dump(result, path)
            return result

        return wrapper

    return decorator


class Diffusion(object):
    """Diffusion class"""

    def __init__(self, features, labels, cache_dir):
        self.features = features
        self.labels = np.array(labels)
        self.N = len(self.features)
        self.cache_dir = cache_dir
        # use ANN for large datasets
        self.use_ann = self.N >= 1_000_000
        if self.use_ann:
            print("ANN creating...")
            self.ann = ANN(self.features, None, method="cosine")
        self.knn = KNN(self.features, None, method="cosine")

    # @cache("offline.jbl")
    def get_offline_results(self, n_trunc, kd=50):
        """Get offline diffusion results for each gallery feature"""
        print("[offline] starting offline diffusion")
        print("[offline] 1) prepare Laplacian and initial state")
        global trunc_ids, trunc_init, lap_alpha
        trunc_ids = None
        trunc_init = None
        lap_alpha = None

        if self.use_ann:
            _, trunc_ids = self.ann.search(self.features, n_trunc)
            sims, ids = self.knn.search(self.features, kd)
            lap_alpha = self.get_laplacian(sims, ids)
        else:
            sims, ids = self.knn.search(self.features, n_trunc)
            trunc_ids = ids
            lap_alpha = self.get_laplacian(sims[:, :kd], ids[:, :kd])
        trunc_init = np.zeros(n_trunc)
        trunc_init[0] = 1
        
        print("[offline] 2) gallery-side diffusion")
        results = Parallel(n_jobs=-1, prefer="threads")(
            delayed(get_offline_result)(i)
            for i in tqdm(range(self.N), desc="[offline] diffusion")
        )
        all_scores = np.concatenate(results)

        print("[offline] 3) merge offline results")
        rows = np.repeat(np.arange(self.N), n_trunc)

        offline = sparse.csr_matrix(
            (all_scores, (rows, trunc_ids.reshape(-1))),
            shape=(self.N, self.N),
            dtype=np.float32,
        )
        return offline

    # @cache('laplacian.jbl')
    def get_laplacian(self, sims, ids, alpha=0.99):
        """Get Laplacian_alpha matrix"""
        affinity = self.get_affinity(sims, ids)
        num = affinity.shape[0]
        degrees = affinity @ np.ones(num) + 1e-12
        # mat: degree matrix ^ (-1/2)
        mat = sparse.dia_matrix(
            (degrees ** (-0.5), [0]), shape=(num, num), dtype=np.float32
        )
        stochastic = mat @ affinity @ mat
        sparse_eye = sparse.dia_matrix(
            (np.ones(num), [0]), shape=(num, num), dtype=np.float32
        )
        lap_alpha = sparse_eye - alpha * stochastic
        return lap_alpha

    # @cache('affinity.jbl')
    def get_affinity(self, sims, ids, gamma=3):
        """Create affinity matrix for the mutual kNN graph of the whole dataset
        Args:
            sims: similarities of kNN
            ids: indexes of kNN
        Returns:
            affinity: affinity matrix
        """
        num = sims.shape[0]
        sims[sims < 0] = 0  # similarity should be non-negative
        sims = sims**gamma
        # vec_ids: feature vectors' ids
        # mut_ids: mutual (reciprocal) nearest neighbors' ids
        # mut_sims: similarites between feature vectors and their mutual nearest neighbors
        vec_ids, mut_ids, mut_sims = [], [], []
        for i in range(num):
            # check reciprocity: i is in j's kNN and j is in i's kNN when i != j
            ismutual = np.isin(ids[ids[i]], i).any(axis=1)
            ismutual[0] = False
            if ismutual.any():
                vec_ids.append(i * np.ones(ismutual.sum(), dtype=int))
                mut_ids.append(ids[i, ismutual])
                mut_sims.append(sims[i, ismutual])
        vec_ids, mut_ids, mut_sims = map(np.concatenate, [vec_ids, mut_ids, mut_sims])
        affinity = sparse.csc_matrix(
            (mut_sims, (vec_ids, mut_ids)), shape=(num, num), dtype=np.float32
        )
        return affinity


def offline_diffusion_search(
    queries,
    train_embeddings,
    labels,
    truncation_size=1000,
    kd=100,
    cache_dir="./cache",
):
    """
    Args:
        queries: predicted embeddings
        gallery: train embeddings
        cache_dir: Directory to cache embeddings
        truncation_size: Number of images in the truncated gallery
        kd: top k results
    """
    n_query = len(queries)
    diffusion = Diffusion(
        np.vstack([queries, train_embeddings]),
        labels,
        cache_dir=cache_dir,
    )
    offline = diffusion.get_offline_results(truncation_size, kd)
    features = preprocessing.normalize(offline, norm="l2", axis=1)
    scores = features[:n_query] @ features[n_query:].T

    ranks = np.argsort(-scores.toarray())[:, :kd]
    return ranks

# Model

In [5]:
class BasicNet(nn.Module):
    def __init__(self, output_features_size=512):
        super().__init__()
        self.output_features_size = output_features_size
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                512, 8, dim_feedforward=2048, dropout=0.2, batch_first=True
            ),
            num_layers=3
        )
        self.avg_pooling = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        x = x.permute((0, 2, 1))
        x = self.transformer(x.float())
        x = x.permute((0, 2, 1))
        x = self.avg_pooling(x).squeeze()
        return x
    
    
def get_backbone(embed_dim):
    net = BasicNet(embed_dim)
    pooling = nn.Identity()
    return net, pooling, embed_dim
    
    
def str_to_bool(condition):
    if isinstance(condition, str):
        if condition.lower() == 'true':
            condition = True
        if condition.lower() == 'false':
            condition = False
    return condition
    
    
class RetrievalNet(nn.Module):
    def __init__(
        self,
        h_dim,
        embed_dim=512,
        norm_features=False,
        without_fc=False,
        with_autocast=False,
        pooling="default",
        projection_normalization_layer="none",
        pretrained=True,
        weights=None,
    ):
        super().__init__()

        norm_features = str_to_bool(norm_features)
        without_fc = str_to_bool(without_fc)
        with_autocast = str_to_bool(with_autocast)

        assert isinstance(without_fc, bool)
        assert isinstance(norm_features, bool)
        assert isinstance(with_autocast, bool)
        self.norm_features = norm_features
        self.without_fc = without_fc
        self.with_autocast = with_autocast
        if with_autocast:
            print("Using mixed precision")

        self.backbone, default_pooling, out_features = get_backbone(h_dim)
        if pooling == "default":
            self.pooling = default_pooling
        elif pooling == "none":
            self.pooling = nn.Identity()
        elif pooling == "max":
            self.pooling = nn.AdaptiveMaxPool1d(output_size=1)
        elif pooling == "avg":
            self.pooling = nn.AdaptiveAvgPool1d(output_size=1)

        if self.norm_features:
            self.standardize = nn.LayerNorm(out_features, elementwise_affine=False)
        else:
            self.standardize = nn.Identity()

        if not self.without_fc:
            self.fc = create_projection_head(
                out_features, embed_dim, projection_normalization_layer
            )
        else:
            self.fc = nn.Identity()

    def forward(self, X):
        with torch.cuda.amp.autocast(enabled=self.with_autocast):
            X = self.backbone(X)
            X = self.pooling(X)
            X = X.view(X.size(0), -1)
            X = self.standardize(X)
            X = self.fc(X)
            X = F.normalize(X, p=2, dim=1)
            return X

# Data

In [6]:
class AudioEmbDataset(Dataset):
    def __init__(
        self,
        data_dir,
        csv_path,
        crop_size=60,
        mode="train",
    ):
        self.get_fn = self._load_item

        self.data_dir = data_dir
        self.mode = mode
        self.crop_size = crop_size
        self.id_encoder = preprocessing.LabelEncoder()

        self.meta_info = pd.read_csv(csv_path, sep="\t")

        if mode != "submission":
            self.meta_info["label"] = self.id_encoder.fit_transform(self.meta_info["artistid"])
            self.classes_ = self.id_encoder.classes_
            with open("classes.txt", "w") as f:
                f.write("\n".join(map(str, self.classes_)))
            if "stage" in self.meta_info.columns:
                self.meta_info = self.meta_info.loc[self.meta_info.stage == mode]
            self.labels = self.meta_info.label.values
        else:
            self.labels = None

        self.paths = self.meta_info.archive_features_path.values
        self.track_ids = self.meta_info.trackid.values

        if mode != "submission":
            self.__get_instance_dict()
    
    def __get_instance_dict(self,):
        self.instance_dict = {cl: [] for cl in set(self.labels)}
        for idx, cl in enumerate(self.labels):
            self.instance_dict[cl].append(idx)
            
    def __process_features(self, x):
        x_len = x.shape[-1]
        if x_len > self.crop_size:
            start = np.random.randint(0, x_len - self.crop_size)
            x = x[..., start : start + self.crop_size]
        else:
            if self.mode == "train":
                i = np.random.randint(0, self.crop_size - x_len) if self.crop_size != x_len else 0
            else:
                i = (self.crop_size - x_len) // 2
            pad_patern = (i, self.crop_size - x_len - i)
            x = torch.nn.functional.pad(x, pad_patern, "constant").detach()
        x = (x - x.mean()) / x.std()
        return x


    def _load_item(self, idx):
        track_features_file_path = self.paths[idx]
        track_features = torch.from_numpy(np.load(
            os.path.join(self.data_dir, track_features_file_path)
        ))
        track_features = self.__process_features(track_features)

        if self.labels is not None:
            label = self.labels[idx]
            label = torch.tensor([label])

            out = {
                "features": track_features,
                "label": label,
                "track_id": self.track_ids[idx],
            }
        else:
            out = {
                "features": track_features,
                "track_id": self.track_ids[idx],
            }
            
        return out

    def __len__(self,):
        return len(self.paths)

    def __getitem__(self, idx):
        return self.get_fn(idx)

# Validation

In [7]:
# INIT MODELS
models = []
for i in range(cfg.N_FOLDS):
    weights_path = os.path.join(cfg.CHECKPOINT_PATH, f"fold{i}.ckpt")
    model = RetrievalNet(
        512,
        512,
        norm_features=True,
        without_fc=True,
        with_autocast=False
    )
    model.load_state_dict(torch.load(weights_path)["net_state"])    
    model.to(cfg.DEVICE)
    model.eval()
    models.append(model)

# Load data
meta_info = pd.read_csv(cfg.TRAIN_CSV_PATH, sep="\t")
test_meta_info = pd.read_csv(cfg.TEST_CSV_PATH, sep="\t")
validation_meta_info = meta_info[meta_info.stage == "test"].reset_index(drop=True)
print("Loaded data")
print("Validation set size: {}".format(len(validation_meta_info)))
print("Test set size: {}".format(len(test_meta_info)))
print()

# Validation
print("Validation")
valid_ds = AudioEmbDataset(cfg.TRAIN_DATA_DIR, cfg.TRAIN_CSV_PATH, cfg.CROP_SIZE, "test")
valid_loader = DataLoader(
    valid_ds,
    batch_size=cfg.BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

embeds = inference(models, valid_loader)
submission = get_ranked_list_diffusion(embeds, 100)
score = eval_submission(submission, validation_meta_info)

print(f"nDCG: {score}")

Loaded data
Validation set size: 16644
Test set size: 41377

Validation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:20<00:00,  1.62it/s]


[offline] starting offline diffusion
[offline] 1) prepare Laplacian and initial state
[offline] 2) gallery-side diffusion


[offline] diffusion: 100%|████████████████████████████████████████████████████████████████████████████████| 33288/33288 [00:26<00:00, 1275.50it/s]


[offline] 3) merge offline results
100 100 100.0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 16644/16644 [00:01<00:00, 8347.74it/s]

nDCG: 0.6848619341470749


# Submission

In [10]:
print("Submission")
test_ds = AudioEmbDataset(cfg.TEST_DATA_DIR, cfg.TEST_CSV_PATH, cfg.CROP_SIZE, "submission")
test_loader = DataLoader(
    test_ds,
    batch_size=cfg.BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

embeds = inference(models, test_loader)
submission = get_ranked_list_diffusion(embeds, 100)
save_submission(submission, cfg.SUBMISSION_PATH)

Submission


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:47<00:00,  1.72it/s]


[offline] starting offline diffusion
[offline] 1) prepare Laplacian and initial state
[offline] 2) gallery-side diffusion


[offline] diffusion: 100%|████████████████████████████████████████████████████████████████████████████████| 82754/82754 [01:04<00:00, 1278.14it/s]


[offline] 3) merge offline results
99 100 99.99997583198395
